In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from func import *

In [ ]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

Main.include("../local_flow_betweenness.jl");
Main.include("../agentSEIR.jl");

In [ ]:
# Read graph
n = 10000
edgelist = np.loadtxt("../data/LFR/LFR_edgelist.txt", dtype=int).tolist()
G = nx.Graph()
G.add_nodes_from(range(1,n+1))
G.add_edges_from(edgelist)

comms = []
with open("../data/LFR/LFR_communities.txt", 'r') as f:
    for line in f.readlines():
        comms.append(list(map(int, line.split(" ")[:-1])))

In [ ]:
len(comms)

In [ ]:
# Identify cut edges between different communities
node_labels = dict()
for l,c in enumerate(comms):
    for v in c:
        node_labels[v] = l+1

cut_edges = set()
for i,j in edgelist:
    if node_labels[i] != node_labels[j]:
        cut_edges.add((i,j))
        cut_edges.add((j,i))

In [ ]:
shortest_path_betweenness = nx.edge_betweenness_centrality(G)
current_flow_betweenness = nx.edge_current_flow_betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

In [ ]:
edges = []
for i,j in edgelist:
    edges.append((i,j))

local50_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), edges, locality_index=.5)
local10_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), edges, locality_index=.1)
local02_betweenness = Main.local_flow_betweenness(np.array(G.nodes()), edges, locality_index=.02)

In [ ]:
def cut_coverage(cut_edges, edge_betweenness, target_perc):
    edges_sorted = sort_betweenness(edge_betweenness)
    target_edges = get_target_edges(edges_sorted, target_perc)
    num = 0
    for e in target_edges:
        if e in cut_edges:
            num += 1
    return num/len(cut_edges)*2

def connected_components(graph, edge_betweenness, target_perc):
    G = graph.copy()
    edges_sorted = sort_betweenness(edge_betweenness)
    target_edges = get_target_edges(edges_sorted, target_perc)
    for u,v in target_edges:
        G.remove_edge(u,v)
    return nx.number_connected_components(G)

In [ ]:
n = G.number_of_nodes()

# Create list of lists representation
ll = [[] for _ in range(n)]
for i in range(1,n+1):
    for j in G.neighbors(i):
            ll[i-1].append(int(j))

In [ ]:
beta = 0.105

# Verify the choice for beta results in ~85% final size
_, sum_S, sum_E, sum_I, sum_R = Main.agentSEIR(ll, days=500, beta=beta)
print(f'Total infection: {sum_R[-1]/n:.4f}')

In [ ]:
reduced_weight = .1
target_perc = np.arange(.01,.31, .01)
num_scenarios = len(target_perc)
num_trials = 50
t_end = 1000
m = G.number_of_edges()

maxCases_ni = np.zeros((num_trials,num_scenarios))
maxCases_ui = np.zeros((num_trials,num_scenarios))
maxCases_hd = np.zeros((num_trials,num_scenarios))
maxCases_eg = np.zeros((num_trials,num_scenarios))
maxCases_sp = np.zeros((num_trials,num_scenarios))
maxCases_rw = np.zeros((num_trials,num_scenarios))
maxCases_lf50 = np.zeros((num_trials,num_scenarios))
maxCases_lf10 = np.zeros((num_trials,num_scenarios))
maxCases_lf02 = np.zeros((num_trials,num_scenarios))
totalCases_ni = np.zeros((num_trials,num_scenarios))
totalCases_ui = np.zeros((num_trials,num_scenarios))
totalCases_hd = np.zeros((num_trials,num_scenarios))
totalCases_eg = np.zeros((num_trials,num_scenarios))
totalCases_sp = np.zeros((num_trials,num_scenarios))
totalCases_rw = np.zeros((num_trials,num_scenarios))
totalCases_lf50 = np.zeros((num_trials,num_scenarios))
totalCases_lf10 = np.zeros((num_trials,num_scenarios))
totalCases_lf02 = np.zeros((num_trials,num_scenarios))

for k in range(num_scenarios):
    
    perc = target_perc[k]
    
    lw_ui = [[1-(1-reduced_weight)*perc]*len(list_) for list_ in ll]
    lw_hd = create_list_of_weights_degree_dist(ll, G, perc, weight=reduced_weight)
    lw_eg = create_list_of_weights_node_centrality(ll, eigenvector_centrality, perc, m, weight=reduced_weight)
    lw_sp = create_list_of_weights(ll, shortest_path_betweenness, perc, weight=reduced_weight)
    lw_rw = create_list_of_weights(ll, current_flow_betweenness, perc, weight=reduced_weight)
    lw_lf50 = create_list_of_weights(ll, local50_betweenness, perc, weight=reduced_weight)
    lw_lf10 = create_list_of_weights(ll, local10_betweenness, perc, weight=reduced_weight)
    lw_lf02 = create_list_of_weights(ll, local02_betweenness, perc, weight=reduced_weight)
    
    for trial in range(num_trials):
        
        print(f'perc = {perc:.2f}, trial {trial+1:d} of {num_trials}', end="\n")
        
        init_rand_infectious = np.random.choice(np.arange(1,n+1), size=int(np.floor(.001*n)), replace=False)
        _, _, sum_E_ni, sum_I_ni, sum_R_ni = Main.agentSEIR(ll, days=t_end, beta=beta, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_ui, sum_I_ui, sum_R_ui = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_ui, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_hd, sum_I_hd, sum_R_hd = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_hd, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_eg, sum_I_eg, sum_R_eg = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_eg, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_sp, sum_I_sp, sum_R_sp = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_sp, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_rw, sum_I_rw, sum_R_rw = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_rw, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf50, sum_I_lf50, sum_R_lf50 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf50, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf10, sum_I_lf10, sum_R_lf10 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf10, rand_init=False, init_infectious_nodes=init_rand_infectious)
        _, _, sum_E_lf02, sum_I_lf02, sum_R_lf02 = Main.agentSEIR(ll, days=t_end, beta=beta, weighted=True, lw=lw_lf02, rand_init=False, init_infectious_nodes=init_rand_infectious)
        
        
        maxCases_ni[trial,k] = max(sum_E_ni + sum_I_ni)
        maxCases_ui[trial,k] = max(sum_E_ui + sum_I_ui)
        maxCases_hd[trial,k] = max(sum_E_hd + sum_I_hd)
        maxCases_eg[trial,k] = max(sum_E_eg + sum_I_eg)
        maxCases_sp[trial,k] = max(sum_E_sp + sum_I_sp)
        maxCases_rw[trial,k] = max(sum_E_rw + sum_I_rw)
        maxCases_lf50[trial,k] = max(sum_E_lf50 + sum_I_lf50)
        maxCases_lf10[trial,k] = max(sum_E_lf10 + sum_I_lf10)
        maxCases_lf02[trial,k] = max(sum_E_lf02 + sum_I_lf02)
        totalCases_ni[trial,k] = sum_R_ni[-1]
        totalCases_ui[trial,k] = sum_R_ui[-1]
        totalCases_hd[trial,k] = sum_R_hd[-1]
        totalCases_eg[trial,k] = sum_R_eg[-1]
        totalCases_sp[trial,k] = sum_R_sp[-1]
        totalCases_rw[trial,k] = sum_R_rw[-1]
        totalCases_lf50[trial,k] = sum_R_lf50[-1]
        totalCases_lf10[trial,k] = sum_R_lf10[-1]
        totalCases_lf02[trial,k] = sum_R_lf02[-1]

In [ ]:
totalCases_ni_mean = np.mean(totalCases_ni/n, axis=0)
totalCases_ui_mean = np.mean(totalCases_ui/n, axis=0)
totalCases_hd_mean = np.mean(totalCases_hd/n, axis=0)
totalCases_eg_mean = np.mean(totalCases_eg/n, axis=0)
totalCases_sp_mean = np.mean(totalCases_sp/n, axis=0)
totalCases_rw_mean = np.mean(totalCases_rw/n, axis=0)
totalCases_lf50_mean = np.mean(totalCases_lf50/n, axis=0)
totalCases_lf10_mean = np.mean(totalCases_lf10/n, axis=0)
totalCases_lf02_mean = np.mean(totalCases_lf02/n, axis=0)

plt.figure(figsize=(9.3,4.5))
plt.plot(target_perc, totalCases_ni_mean, label='NI', linestyle='-', color='dimgray', linewidth=6)
plt.plot(target_perc, totalCases_ui_mean, label='UI', linestyle=(0,(5,5)), color='k', linewidth=6)
plt.plot(target_perc, totalCases_hd_mean, label='HD', linestyle=(0,(3,5,1,5)), color='tab:red', linewidth=6)
plt.plot(target_perc, totalCases_eg_mean, label='EG', linestyle=(0,(3,1,1,1)), color='tab:brown', linewidth=6)
plt.plot(target_perc, totalCases_sp_mean, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, totalCases_rw_mean, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot(target_perc, totalCases_lf50_mean, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, totalCases_lf10_mean, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, totalCases_lf02_mean, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.5, 1.4), ncol=3, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.01, .05, .1, .15, .2, .25, .3), ('$1\%$', '$5\%$', '$10\%$', '$15\%$', '$20\%$', '$25\%$', '$30\%$'), color='k', size=18)
plt.yticks((.0, .2, .4, .6, .8),size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Final Epidemic Size',fontsize=22)
plt.savefig("LFR_episize_randinit.png", bbox_inches='tight', format='png', dpi=400)
plt.show()

In [ ]:
target_perc = np.arange(.01,.31, .01)

sp_cut_coverage = []
cf_cut_coverage = []
lf50_cut_coverage = []
lf10_cut_coverage = []
lf02_cut_coverage = []

for perc in target_perc:
    sp_cut_coverage.append(cut_coverage(cut_edges, shortest_path_betweenness, perc))
    cf_cut_coverage.append(cut_coverage(cut_edges, current_flow_betweenness, perc))
    lf50_cut_coverage.append(cut_coverage(cut_edges, local50_betweenness, perc))
    lf10_cut_coverage.append(cut_coverage(cut_edges, local10_betweenness, perc))
    lf02_cut_coverage.append(cut_coverage(cut_edges, local02_betweenness, perc))

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

fig, ax = plt.subplots(figsize=(9.3,5.5))


plt.plot(target_perc, sp_cut_coverage, label='SP', linestyle=(0,(5,1)), color='tab:orange', linewidth=6)
plt.plot(target_perc, cf_cut_coverage, label='CF', linestyle='dashed', color='tab:green', linewidth=6)
plt.plot([], [], label=' ', linestyle=(0,(1,1)), color="w", linewidth=0)
plt.plot(target_perc, lf50_cut_coverage, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=6)
plt.plot(target_perc, lf10_cut_coverage, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=6)
plt.plot(target_perc, lf02_cut_coverage, label='LF(1/50)', linestyle='dashdot', color='tab:blue', linewidth=6)
leg = plt.legend(fontsize=18, bbox_to_anchor=(.68, .35), ncol=2, loc='upper center', handlelength=4)
for i in leg.legendHandles:
    i.set_linewidth(4)
plt.xticks((.01, .05, .1, .15, .2, .25, .3), ('$1\%$', '$5\%$', '$10\%$', '$15\%$', '$20\%$', '$25\%$', '$30\%$'), color='k', size=18)
plt.yticks((.0, .2, .4, .6, .8, 1), ('$0\%$', '$20\%$', '$40\%$', '$60\%$', '$80\%$', '$100\%$'), size=18)
plt.xlabel('Percentage of Targeted Edges', fontsize=22)
plt.ylabel('Percentage of Targeted Cuts',fontsize=22)


axins = zoomed_inset_axes(ax, 4, loc=5) # zoom = 10
axins.plot(target_perc, lf50_cut_coverage, label='LF(1/2)', linestyle=(0,(1,1)), color="tab:cyan", linewidth=4)
axins.plot(target_perc, lf10_cut_coverage, label='LF(1/10)', linestyle=(0,(3,1,1,1,1,1)), color='tab:purple', linewidth=4)
axins.plot(target_perc, cf_cut_coverage, linestyle='dashed', color='tab:green', linewidth=4)
axins.plot(target_perc, lf02_cut_coverage, linestyle='dashdot', color='tab:blue', linewidth=4)
axins.set_xlim(.13, .17)
axins.set_ylim(.95, 1.015)
axins.set_xticklabels('')
axins.set_yticklabels('')
axins.tick_params(axis = "both", which = "both", bottom = False, left = False)
mark_inset(ax, axins, loc1=2, loc2=1, fc="none", ec="0")



plt.savefig("LFR_cut_edges.png", bbox_inches='tight', format='png', dpi=400)
plt.show()

